In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np 
import pandas as pd 
import seaborn as sns; sns.set(style="ticks", color_codes=True)
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import seaborn as sns
import plotly.express as px
%matplotlib inline

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected = True)

In [ ]:
NA_list=['NaN', 'Null', 'null serotype' ]
df=pd.read_csv('/kaggle/input/disease-outbreaks-in-nigeria-datasets/meningitis_dataset.csv', na_values=NA_list)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
df

In [ ]:
# how many dieseas are there in data?

df.disease.value_counts()

In [ ]:
data=df[['id', 'gender', 'state', 'settlement', 'report_year', 'age', 'disease', 'serotype', 
         'health_status', 'report_outcome']]
data.head()

In [ ]:
data.isnull().sum()

we can with aid of LabelEncoder later dedicate Null-Values to a separtae Class.

# Data Visualization

Diseases and states or settelment

In [ ]:
data_state= data.groupby(['state', 'disease'])['id'].count().reset_index(name='Number of Patients').sort_values(by= 'Number of Patients', ascending= False)
data_state

px.bar(data_state, x= 'state' , y='Number of Patients', color= 'disease')


In [ ]:
data_settlement= data.groupby(['settlement', 'disease', 'health_status'])['id'].count().reset_index(name='Number of Patients').sort_values(by= 'Number of Patients', ascending= False)
data_settlement

px.bar(data_settlement, x= 'settlement' , y='Number of Patients', color= 'disease')

In [ ]:
data_year= data.groupby(['state', 'report_year'])['id'].count().reset_index(name='Number of Patients').sort_values(by= 'Number of Patients', ascending= False)
data_year

px.bar(data_year, x= 'report_year', y='Number of Patients', color= 'state')


# Diseases

In [ ]:
data_disease= data.groupby(['disease'])['id'].count().reset_index(name='Number of Patients').sort_values(by= 'Number of Patients', ascending= False)
data_state
px.pie(data_disease, values='Number of Patients', names= 'disease' , title= 'Number of patients of each dieases', 
       width=800 , height=800 , color_discrete_sequence = px.colors.sequential.RdBu)

number of patinets in each year

In [ ]:
data_years= data.groupby(['report_year'])['id'].count().reset_index(name='Number of Patients').sort_values(by= 'Number of Patients', ascending= False)
data_years
px.pie(data_years, values='Number of Patients', names= 'report_year' , title= 'Number of patients in each year', 
       width=800 , height=800 , color_discrete_sequence = px.colors.sequential.RdBu)

# Disease and gender of patinets

In [ ]:
data_gender= data.groupby(['gender', 'disease', 'health_status'])['id'].count().reset_index(name='Number of Patients').sort_values(by= 'Number of Patients', ascending= False)
data_gender

px.bar(data_gender, x= 'gender' , y='Number of Patients', color= 'disease')


In [ ]:
px.bar(data_gender, x= 'gender' , y='Number of Patients', color= 'health_status')


In [ ]:
data1=data.loc[data['health_status']== 'Alive']
data1.gender.value_counts()

In [ ]:
data1=data.loc[data['health_status']== 'Dead']
data1.gender.value_counts()

the features have no correlation with dieseas or with eachothers!!!!

# dieseas and age of Patients

In [ ]:
data_age= data.groupby(['age', 'disease', 'health_status'])['id'].count().reset_index(name='Number of Patients').sort_values(by= 'Number of Patients', ascending= False)
data_age

px.bar(data_age, x= 'age' , y='Number of Patients', color= 'health_status')


In [ ]:
px.bar(data_age, x= 'health_status' , y='Number of Patients', color= 'age')


In [ ]:
data_age2= data.groupby(['age', 'disease' ])['id'].count().reset_index(name='Number of Patients').sort_values(by= 'Number of Patients', ascending= False)
data_age2


In [ ]:
px.scatter(data_age2, x= 'age', y='Number of Patients' , color='disease' )

In [ ]:
data_serotype= data.groupby(['serotype', 'disease' ])['id'].count().reset_index(name='Number of Patients').sort_values(by= 'Number of Patients', ascending= False)
data_serotype

In [ ]:
px.bar(data_serotype, x='serotype' , y='Number of Patients', color= 'disease' )


Conclusion:

we can colculde from Data-Visulization that dieseases can occur independent of state, settlement, gender. and maybe only age is a feature which can affect the health-Status

In [ ]:
data.health_status.value_counts()

In [ ]:

data.loc[df['health_status']== 'alive', 'health_status']='Alive'        

In [ ]:
data.health_status.value_counts()

In [ ]:
data.loc[df['report_outcome']== 'confirmed', 'report_outcome']='Confirmed'     

In [ ]:
data.report_outcome.value_counts()

In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
def Labelencoder_feature(x):
    le=LabelEncoder()
    x=le.fit_transform(x)
    return x
data2=data.drop(columns=['id'])
data2=data2.apply(Labelencoder_feature)
data2.head()

In [ ]:
fig = plt.figure(figsize=(20,10))
sns.heatmap(data2.corr(), annot = True, fmt = '.2f', cmap = 'Blues', )

In [ ]:
Features_Region_p=sns.PairGrid(data2, hue='disease')

Features_Region_p.map_diag(plt.hist)
Features_Region_p.map_offdiag(plt.scatter)

the only feature which may have a correlation with dieseas is serotype and we see it is why this feature has values only for a spesific disease (Meninigent)
there is no reason to continue forecasting with aid of these features. Diseases have no colrelation with other features 